##### Sesión 15: Caso Práctico

## Modelo Segmentación K means - Consumo de clientes con su tarjeta  


### Objetivo:

Dado que los clientes reciben diversas ofertas de consumo para que estos puedan transaccionar con su tarjeta de crédito y débito, la entidad bancaria no está segura si sus clientes tienen algún interes en sus ofertas, teniendo en cuenta que cada oferta ya corresponde un gasto para la entidad.

Se pide realizar un estudio de segmentación para conocer cuáles son las preferencias de consumo que sus clientes optarían para comunicarles ofertas más direccionadas.

* Puede guiarse del procedimiento del caso resuelto de K-means 
* Previamente corra el código en este notebook para preparar la data
* El dataset consta más de 300K operaciones por cliente
* El dataset incluye información de 12 meses de historia
* La data consta de 21 rubros de consumo:

        1.	Prodsuper:		grupo supermercados
        2.	Restbar:		grupo restaurantes y bares
        3.	Salud:		grupo farmacia, clínicas, seguro salud, masaje, etc.
        4.	Vehrep:		grupo repuestos, compra vehículo, gasolina, etc.
        5.	Entretenimiento:	grupo entrenamiento, guía turística.
        6.	Tiendadepar:	grupo tienda por departamento.
        7.	Ropamoda:		grupo tienda de ropa, tienda independiente de mall
        8.	Prodpersondiv:	grupo diversos productos personales, joyería, etc.
        9.	Telcom:		grupo telecomunicaciones, pago celular, recargas, etc.
        10.	Financiero:		grupo financiero, pago de impuestos, seguros, etc.
        11.	Transplaerea:	grupo transporte, aerolíneas, bus interprovincial, etc.
        12.	Clubmkt:		grupo clubes
        13.	Prodlocal:		grupo productos locales
        14.	Enseñanza:		grupo enseñanza, pago universidades, academia, etc.
        15.	Belleza:		grupo belleza, gimansio, maquillaje, etc.
        16.	Prodelectro:	grupo producto electrónico.
        17.	Alqbienes:		grupo alquiler de bienes, hoteles, pago de departamento, etc
        18.	Artcultura:		grupo arte y cultura, teatro, librería, etc.
        19.	Profdiverso:	grupo profesional, servicio legal, consultoría, etc.
        20.	Hogaroficina:	grupo hogar y oficina, utensilios para la oficina u hogar.
        21.	Informática:	grupo informática, software, reparación Pc, etc.          


## 1. Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 10)

## 2. Carga y preparación de la base

In [ ]:
dataFramePre = pd.read_csv("data/02dataBaseConsumo.txt",delimiter='|', encoding='latin-1')
display(dataFramePre.head())

Realizaremos la construcción de nuestra Matriz de Segmentación, llevando nuestra base de datos a nivel de ***cliente*** y creando variables de rubro de consumo, donde agruparemos por la cantidad de transacciones.

In [ ]:
# Reinicio de índice "df.reset_index()"
copy=dataFramePre
dataFrame = pd.DataFrame() 

dataFrame['trxGrupoGiro']=copy.groupby(["cliente","grupoGiro","edad","ingreso","sexo"]).agg("trx").sum()
dataFrame = pd.pivot_table(dataFrame,'trxGrupoGiro',['cliente',"edad","ingreso","sexo"],'grupoGiro')
dataFrame = dataFrame.fillna(0)
dataFrame

In [ ]:
dataFrame = dataFrame.reset_index()
display(dataFrame.head())

## Objetivo

In [ ]:
# Variables objetivo de estudio:
rubroName = ['prodsuper', 'restbar', 'salud', 'vehrep', 'entretenimiento', 'tiendadepar', 'ropamoda', 'prodpersondiv',
               'telcom','financiero', 'transplaerea','clubmkt','prodlocal','enseñanza','belleza','prodelectro',
               'alqbienes','artcultura','profdiverso','hogaroficina','informatica']

### Exploración

In [ ]:
# Base con la que se vamos a trabajar
df_final = dataFrame[rubroName]
display(df_final.head(10))

In [ ]:
print(df_final.describe())

In [ ]:
dataFrame[rubroName].hist(bins = 50, figsize=(20,15))
plt.show()

In [ ]:
for columnName in rubroName:
    plt.title(columnName)  
    plt.boxplot(dataFrame[columnName], 0, 'gD')    
    plt.show()

### Transformación (normalización estándar)

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
#scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))

# ************
# Lo aplicamos
# ************
for columnName in rubroName:
    dataFrame[columnName] = scaler.fit_transform(dataFrame[columnName].values.reshape(-1, 1))
    
dataFrame[rubroName].head()

### Outliers

In [ ]:
def calculateNumOutliars(serie):
    mu = serie.mean()
    desv = np.std(serie)
    a = ((serie-mu)/desv < -2) | ((serie-mu)/desv > 2)
    numOutliars = a[a == True].shape[0]
    return a,numOutliars    

In [ ]:
numTotal = dataFrame.shape[0]
for columnName in rubroName:
    a,numOutliars = calculateNumOutliars(dataFrame[columnName])
    # Creamos nuevos campos para filtrar los Outliers 
    dataFrame['flg_'+columnName]=a
    print('*'+columnName)
    if numOutliars > 0:
        print("Número de valores outliars: " + str(numOutliars))
        print("Porcentaje: " + str(np.round(numOutliars * 100 / numTotal, 2)) + "%")
    else:
        print("****No hay Outliers")    
    print("\n")

In [ ]:
dataFrame = dataFrame[(dataFrame['flg_prodsuper']==False)&
                      (dataFrame['flg_restbar']==False)&
                      (dataFrame['flg_salud']==False)&
                      (dataFrame['flg_vehrep']==False)&
                      (dataFrame['flg_entretenimiento']==False)&
                      (dataFrame['flg_tiendadepar']==False)&
                      (dataFrame['flg_ropamoda']==False)&
                      (dataFrame['flg_prodpersondiv']==False)&
                      (dataFrame['flg_telcom']==False)&
                      (dataFrame['flg_financiero']==False)&
                      (dataFrame['flg_transplaerea']==False)&
                      (dataFrame['flg_clubmkt']==False)&
                      (dataFrame['flg_prodlocal']==False)&
                      (dataFrame['flg_enseñanza']==False)&
                      (dataFrame['flg_belleza']==False)&
                      (dataFrame['flg_prodelectro']==False)&
                      (dataFrame['flg_alqbienes']==False)&
                      (dataFrame['flg_artcultura']==False)&
                      (dataFrame['flg_profdiverso']==False)&
                      (dataFrame['flg_hogaroficina']==False)&
                      (dataFrame['flg_informatica']==False)]
dataFrame = dataFrame.reset_index()

print('Cantidad de Registros sin Outliers: '+str(dataFrame.shape[0]))
dataFrame[rubroName].head()

###  Reducción de dimensión (PCA)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(dataFrame[rubroName])
pca.explained_variance_ratio_

In [ ]:
for i in range(len(pca.components_)):
    print('% Var. explicada ('+str(i+1)+' componentes): ', np.cumsum(pca.explained_variance_ratio_)[i]*100)
    
plt.bar(range(1,len(pca.components_)+1),pca.explained_variance_ratio_, alpha=.2,color='0')
plt.plot(range(1,len(pca.components_)+1),np.cumsum(pca.explained_variance_ratio_),alpha=4)
plt.title("Varianza explicada y pareto")
plt.show()

In [ ]:
# Elegimos la componente adecuada:
pcaFin = PCA(n_components=11)
pcaFin.fit(dataFrame[rubroName])
pd.DataFrame(pcaFin.components_,columns=rubroName)

### Modelamiento

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import pairwise_distances_argmin_min 

In [ ]:
# Calculando el número de clúster adecuado:
X = dataFrame[rubroName]
X.head()

In [ ]:
# Calculando el número de clúster adecuado:
X = dataFrame[rubroName]
score = []

for i in range(1, 20):
    kmeans = KMeans(n_clusters=i,max_iter=600, algorithm = 'auto')
    kmeans_model = kmeans.fit(X)
    score.append(kmeans_model.inertia_)

In [ ]:
plt.plot(range(1, 20), score, marker='o')
plt.xlabel('Número de Clúster')
plt.ylabel('Score')
plt.title('Elbow method')
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score

# Nos fijamos de los indicadores de clustering:

for i in range(2,12): 
    kmeans = KMeans(n_clusters=i,max_iter=600, algorithm = 'auto')
    kmeans_model = kmeans.fit(X)
    cluster = kmeans_model.predict(X)
    print(str(i)+' clústeres:')
    print('Inercia: '+str(kmeans.inertia_))
    print('Silueta: '+str(silhouette_score(X, cluster, metric='euclidean', sample_size = 5000, random_state=123)))

In [ ]:
# coriendo para 3 clusters
kmeans = KMeans(n_clusters=3,max_iter=600, algorithm = 'auto')
kmeans_model = kmeans.fit(X)
cluster = kmeans_model.predict(X)
    
# graficando para esta cantidad de clusters
fig = plt.figure()
f1 = dataFrame['restbar'].values
f2 = dataFrame['salud'].values
 
#colores=['red','green','blue','yellow']
colores=['yellow','green','blue']
asignar=[]
for row in cluster:
    asignar.append(colores[row])
    
plt.scatter(f1, f2, c=asignar, marker='*', s=100)
#plt.scatter(centroide[2][:, 0], centroide[2][:, 1], marker='*', c='yellow', s=100)
plt.show()

In [ ]:
fig = plt.figure()
f1 = dataFrame['restbar'].values
f2 = dataFrame['prodsuper'].values
 
#colores=['red','green','blue','yellow']
colores=['yellow','green','blue']
asignar=[]
for row in cluster:
    asignar.append(colores[row])
    
plt.scatter(f1, f2, c=asignar, marker='*', s=100)
plt.show()

### Evaluación

In [ ]:
numClus = [2,3,4,5]

kmeans = [KMeans(n_clusters=i,max_iter=600, algorithm = 'auto') for i in numClus]
score = [kmeans[i].fit(X).score(X) for i in range(len(kmeans))]
cluster = [kmeans[i].predict(X) for i in range(len(kmeans))]

In [ ]:
centroide = [kmeans[i].cluster_centers_ for i in range(len(kmeans))]
copy =  pd.DataFrame()

for i in numClus:
    # Distribución de los grupos por clúster:
    copy['cluster'] = cluster[i-2]   #cuidado con la cantidad de clusters
    cantidadGrupo =  pd.DataFrame()
    cantidadGrupo['ctdCliente']=copy.groupby('cluster').size()
    cantidadGrupo['pctCliente']=round(100*cantidadGrupo['ctdCliente']/cantidadGrupo['ctdCliente'].sum(),2)
    
    # gráfico de los grupos según su distribución:
    plt.pie(cantidadGrupo['pctCliente'], labels=cantidadGrupo.index, autopct='%1.1f%%')
    plt.title('Clúster '+str(i))
    plt.legend()
    plt.show()
    print(cantidadGrupo)       
    print('\n')

In [ ]:
centroide = [kmeans[i].cluster_centers_ for i in range(len(kmeans))]
copy =  pd.DataFrame()

for i in numClus:
    # Distribución de los grupos por clúster:
    copy['cluster'] = cluster[i-2]   #cuidado con la cantidad de clusters
    cantidadGrupo =  pd.DataFrame()
    cantidadGrupo['ctdCliente']=copy.groupby('cluster').size()
    cantidadGrupo['pctCliente']=round(100*cantidadGrupo['ctdCliente']/cantidadGrupo['ctdCliente'].sum(),2)
    
    # gráfico de los grupos según su distribución:
    plt.pie(cantidadGrupo['pctCliente'], labels=cantidadGrupo.index, autopct='%1.1f%%')
    plt.title('Clúster '+str(i))
    plt.legend()
    plt.show()
    print(cantidadGrupo)       
    print('\n')

In [ ]:
numClusFinal = int(input('Ingrese el número de clúster: '))

In [ ]:
dfval = pd.DataFrame(centroide[numClusFinal-2],columns=rubroName)
dfval

In [ ]:
plt.rcParams['figure.figsize'] = (10, 10)
#plt.style.use('ggplot')

dfval.plot.barh()